In [55]:
# importing the necessary packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import csv

In [56]:
search_term1="COVID-19"
search_term2="neurologic manifestations"
# url="https://pubmed.ncbi.nlm.nih.gov/?term=%22{}%22+%22{}%22&size=200".format(search_term1,search_term2) #w/ quotes
url="https://pubmed.ncbi.nlm.nih.gov/?term={search1}+{search2}&size=200".format(search1=search_term1,search2=search_term2) #no quotes
url2="https://pubmed.ncbi.nlm.nih.gov/"

r1 = requests.get(url)
all_papers = r1.content

soup1 = BeautifulSoup(all_papers, 'html5lib')

papers = soup1.find_all('a', class_='labs-docsum-title')

link_abstracts=[]
# link2=url2+papers[1]['href']
# print(link2)
# r2 = requests.get(link2)
# paper_content = r2.content
# soup2 = BeautifulSoup(paper_content, 'html5lib')
# title=soup2.find_all('title')
# print(title)
# keyword=soup2.find_all('p')
# for x in keyword:
#     if x.find(text=re.compile("Keyword")):
#         clean = re.compile('<.*?>')
#         print(re.sub(clean, '', str(x)))
# # print(keyword)

In [57]:
# from https://medium.com/@jorlugaqui/how-to-strip-html-tags-from-a-string-in-python-7cb81a2bbf44
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def remove_line(text):
    text_new = text.replace('\n','')
    return text_new

In [58]:
for paper in papers:
    link=url2+paper['href']
    r2 = requests.get(link)
    paper_content = r2.content
    soup2 = BeautifulSoup(paper_content, 'html5lib')
    abstract = soup2.find_all('div', class_='abstract-content')
    title=soup2.find_all('title')
    keyword=soup2.find_all('p')
    if not bool(abstract):
        abstract_text='No Abstract'
    else: 
        raw_text = str(abstract)
        raw_text2 = remove_html_tags(raw_text)
        abstract_text=remove_line(raw_text2)
    
    keyword_text='No Keywords'
    for x in keyword:
        if x.find(text=re.compile("Keyword")):
            raw_text = str(x)
            raw_text2 = remove_html_tags(raw_text)
            keyword_text=remove_line(raw_text2)

    raw_text = str(title)
    raw_text2 = remove_html_tags(raw_text)
    title_text=remove_line(raw_text2)
    
    link_abstracts.append(tuple([title_text, keyword_text, abstract_text, link]))

In [59]:

filename="COVID-19+neuro_manifestations"
file_ext=filename+".csv"

with open(file_ext, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['title','keyword','abstract','link'])
        for row in link_abstracts:
            writer.writerow(row)
            
print('done')

done
